# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([3,1,4,2,5])
x

tensor([3, 1, 4, 2, 5])

In [3]:
y = x + (2*x)

In [4]:
print(y)

tensor([ 9,  3, 12,  6, 15])


In [5]:
import syft as st

In [6]:
hook = st.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
blob = st.VirtualWorker(hook, id="blob")

In [9]:
blob._objects

{}

In [10]:
x = th.tensor([2,5,3,1,4])

In [11]:
x = x.send(blob)

In [12]:
blob._objects

{56931564131: tensor([2, 5, 3, 1, 4])}

In [13]:
x

(Wrapper)>[PointerTensor | me:80946848538 -> blob:56931564131]

In [14]:
x.location

<VirtualWorker id:blob #tensors:1>

In [15]:
x.id

80946848538

In [16]:
x.id_at_location

56931564131

In [17]:
x.owner

<VirtualWorker id:me #tensors:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #tensors:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:80946848538 -> blob:56931564131]

In [20]:
x = x.get()

In [21]:
x

tensor([2, 5, 3, 1, 4])

In [22]:
blob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [23]:
# try this project here!
alice = st.VirtualWorker(hook, id="alice")


In [24]:
x = th.tensor([2,5,3,6])

In [25]:
x.ptr = x.send(blob,alice)

In [26]:
x.ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:26922888089 -> blob:78692432953]
	-> (Wrapper)>[PointerTensor | me:10905027049 -> alice:67862555]

In [27]:
x.ptr.get()

[tensor([2, 5, 3, 6]), tensor([2, 5, 3, 6])]

In [28]:
x_ptr = th.tensor([2,4,3,1]).send(blob, alice)

In [29]:
x_ptr.get(sum_results = True)

tensor([4, 8, 6, 2])

# Lesson: Introducing Remote Arithmetic

In [30]:
x = th.tensor([1,3,5,2]).send(blob)
y = th.tensor([2,5,3,8]).send(blob)

In [31]:
x

(Wrapper)>[PointerTensor | me:86764296335 -> blob:71346162786]

In [32]:
y

(Wrapper)>[PointerTensor | me:74461192302 -> blob:84988571486]

In [33]:
z = x + y

In [34]:
a = z.get()

In [35]:
a

tensor([ 3,  8,  8, 10])

In [36]:
z = th.add(x,y)

In [37]:
z

(Wrapper)>[PointerTensor | me:25355767013 -> blob:97197127573]

In [38]:
z.get()

tensor([ 3,  8,  8, 10])

In [39]:
x = th.tensor([1.,3,5,2], requires_grad = True).send(blob)
y = th.tensor([2.,5,3,8], requires_grad=True).send(blob)

In [40]:
z = (x + y).sum()

In [41]:
z.backward()

(Wrapper)>[PointerTensor | me:17403004764 -> blob:14260272562]

In [42]:
x = x.get()

In [43]:
x

tensor([1., 3., 5., 2.], requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [44]:
# try this project here!
ip = th.tensor([[1.,1], [0,1,], [1,0], [0,0]],requires_grad = True).send(blob)
target = th.tensor([[1.], [1], [0], [0]], requires_grad = True).send(blob)

In [45]:
weights = th.tensor([[0.], [0.]], requires_grad = True).send(blob)

In [46]:
for i in range(10):
    pred = ip.mm(weights)
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad*0.1)
    weights.grad *= 0 
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [47]:
blob = blob.clear_objects()

In [48]:
blob._objects

{}

In [49]:
x = th.tensor([1,2,3,4,5]).send(blob)

In [50]:
blob._objects

{55630147071: tensor([1, 2, 3, 4, 5])}

In [51]:
del x

In [52]:
blob._objects

{}

In [53]:
x = th.tensor([1,2,3,4,5]).send(blob)

In [54]:
blob._objects

{71843336462: tensor([1, 2, 3, 4, 5])}

In [55]:
x = "asdf"

In [56]:
blob._objects

{}

In [57]:
x = th.tensor([1,2,3,4,5]).send(blob)

In [58]:
x

(Wrapper)>[PointerTensor | me:24429815651 -> blob:19359746662]

In [59]:
blob._objects

{19359746662: tensor([1, 2, 3, 4, 5])}

In [60]:
x = "asdf"

In [61]:
blob._objects

{19359746662: tensor([1, 2, 3, 4, 5])}

In [62]:
del x

In [63]:
blob._objects

{19359746662: tensor([1, 2, 3, 4, 5])}

In [64]:
blob = blob.clear_objects()
blob._objects

{}

In [65]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(blob)

In [66]:
blob._objects

{92837856999: tensor([1, 2, 3, 4, 5])}

In [67]:
x = th.tensor([1,2,3,4,5]).send(blob)
y = th.tensor([1,1,1,1,1])

In [68]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(blob) on the other tensor.

Tensor A: [PointerTensor | me:70376295590 -> blob:59653554077]
Tensor B: tensor([1, 1, 1, 1, 1])

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [69]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(blob) on the other tensor.

Tensor A: [PointerTensor | me:70376295590 -> blob:59653554077]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [71]:
from torch import nn, optim

In [72]:
# A Toy Dataset
data = th.tensor([[1., 1],[0,1],[1,0],[0,0]], requires_grad = True)
target = th.tensor([[1.],[1],[0],[0]], requires_grad = True)

In [73]:
# A Toy Model
model = nn.Linear(2,1)

In [74]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [83]:
def train(iterations = 20):
    for iter in range(iterations):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target)**2).sum()
        loss.backward()
        opt.step()
        print(loss.data)
        
train()

tensor(0.0012)
tensor(0.0009)
tensor(0.0007)
tensor(0.0005)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)
tensor(0.0001)
tensor(0.0001)
tensor(7.9731e-05)
tensor(6.0932e-05)
tensor(4.6569e-05)
tensor(3.5595e-05)
tensor(2.7209e-05)
tensor(2.0800e-05)
tensor(1.5901e-05)
tensor(1.2156e-05)
tensor(9.2939e-06)
tensor(7.1060e-06)


In [85]:
data_blob = data[0:2].send(blob)
target_blob = target[0:2].send(blob)

In [86]:
data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [87]:
datasets = [(data_blob, target_blob), (data_alice, target_alice)]

In [89]:
def train(iterations = 20):
    
    model = nn.Linear(2,1)
    opt = optim.SGD(params = model.parameters(), lr = 0.1)
    
    for iter in range(iterations):
        
        for _data, _target in datasets:
            
            # send the model to the data
            model = model.send(_data.location)
            
            #resume training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()
            
            # get back smarter model
            model = model.get()
            
            print(loss.get())

train()

tensor(0.3205, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0005, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0003, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0002, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(0.0001, requires_grad=True)
tensor(9.3918e-05, requires_grad=True)
tensor(7.8169e-05, requires_grad=True)
tensor(6.5060e-05, requires_grad=True)
tensor(5.4150e-05, requires_grad=True)
tensor(4.5070e-05, requires_grad=True)
tensor(3.7512e-05, requires_grad=True)
tensor(3.1222e-05, requires_grad=True)
tensor(2.5987e-05, requires_grad=True)
tensor(2.1629e-05, requires_grad=True)
tensor(1.8002e-05, requires_grad=True)
tensor(1.4984e-05, requires_grad=True)
tensor(1.2470e-05, requires_grad=True)
tensor(1.0380e-05, requires_grad=True)
tensor(8.6392e-06, requires_grad=True)
tensor(7.1904e-06, requires_grad=True)
tensor(5.9842e-06, requires_gr

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [163]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:bob #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [167]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [170]:
x = x.send(alice)

In [171]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5])}

In [172]:
alice._objects

{55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [174]:
y = x + x

In [175]:
y

(Wrapper)>[PointerTensor | me:48924169518 -> alice:48924169518]

In [176]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5]),
 48924169518: tensor([ 2,  4,  6,  8, 10])}

In [177]:
alice._objects

{48924169518: (Wrapper)>[PointerTensor | alice:48924169518 -> bob:48924169518],
 55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [178]:
jon = sy.VirtualWorker(hook, id="jon")

In [186]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [187]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [188]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [189]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40541026396 -> bob:10539507281]

In [190]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [191]:
alice._objects

{}

In [192]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [193]:
bob._objects

{}

In [194]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [195]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [196]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [197]:
del x

In [198]:
bob._objects

{}

In [199]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [212]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [213]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [214]:
bob._objects

{81966670653: tensor([1, 2, 3, 4, 5])}

In [215]:
alice._objects

{}

In [216]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [217]:
bob._objects

{}

In [218]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

In [202]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [203]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [204]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

In [205]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [206]:
bob._objects

{}

In [207]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [208]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [209]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [210]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [211]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}